# Census Data for San Antonio Districts

In [1]:
import pandas as pd
import numpy as np
import os
import re
import math

## District Data

This sheet tells which district each census tract aligns with and what percentage of the district is included. District 0 are census tracts outside of San Antonio.

In [2]:
dist = pd.read_excel('CMO_CDTractsPrecincts_220406.xlsx')
dist = dist[['Council District',
             'Tract 2020 ID\n(Use with 2020 Decennial Census)',
             'Percentage of Tract 2020 in Council District']]
dist.columns = ['council','tract','percent']
dist = dist.iloc[1:].reset_index(drop=True)
dist.tract = dist.tract.astype(str)
dist.council = np.where(dist.council == 'Outside CoSA', 0, dist.council)
dist.head()

,council,tract,percent
0,1,110100,0.999534
1,1,110300,0.499153
2,1,110500,0.000150
3,1,110600,0.002611
4,1,110700,0.953113


## Census Data

This is all the data pulled from the census. 

In [3]:
filename = 'ACSDP5Y2021.DP03-Data.csv'
filename2 = 'ACSDP5Y2021.DP03-Column-Metadata.csv'

In [4]:
dataall = pd.read_csv(filename)
dataall.columns = dataall.columns.str.lower()

dataall = dataall.iloc[1:].reset_index(drop=True)
dataall = dataall.replace('(X)', np.nan).replace('-',np.nan).replace('\*+',np.nan,regex=True).replace('N',np.nan)
# dataall = dataall.dropna(axis=1, how='all')

dataall.geo_id = dataall.geo_id.str[-6:]
dataall.geo_id = dataall.geo_id.astype(str)
dataall = dataall.drop(columns='name')

to_fix = []
for col in dataall.columns:
    bad_num = dataall[col].astype(str).str.contains('+',regex=False).sum()
    if bad_num > 0:
        to_fix.append(col)
for col in to_fix:
    dataall[col] = dataall[col].astype(str)\
    .str.replace('+','',regex=False)\
    .str.replace(',','').astype(float)

dataall.head()

,geo_id,dp03_0001e,dp03_0001m,dp03_0001ma,dp03_0001ea,dp03_0002e,dp03_0002ea,dp03_0002m,dp03_0002ma,dp03_0003e,...,dp03_0135pma,dp03_0136pe,dp03_0136pm,dp03_0136pma,dp03_0136pea,dp03_0137pe,dp03_0137pm,dp03_0137pma,dp03_0137pea,unnamed: 1098
0,110100,2779,583,NaN,NaN,1778,NaN,430,NaN,1715,...,NaN,12.1,19.1,NaN,NaN,21.7,7.8,NaN,NaN,NaN
1,110300,2327,455,NaN,NaN,1718,NaN,433,NaN,1718,...,NaN,5.0,6.8,NaN,NaN,32.7,12.3,NaN,NaN,NaN
2,110500,1376,175,NaN,NaN,679,NaN,128,NaN,662,...,NaN,65.3,8.8,NaN,NaN,73.3,12.8,NaN,NaN,NaN
3,110600,5152,1603,NaN,NaN,939,NaN,216,NaN,939,...,NaN,41.3,19.6,NaN,NaN,50.5,13.8,NaN,NaN,NaN
4,110700,878,204,NaN,NaN,434,NaN,196,NaN,434,...,NaN,25.0,13.5,NaN,NaN,66.0,18.2,NaN,NaN,NaN


## Labels

These are the labels for the census data.

In [5]:
labels = pd.read_csv(filename2)
labels.columns = labels.columns.str.lower().str.replace('\W+','_', regex=True)
labels.column_name = labels.column_name.str.lower()

labels = labels [labels.label.str.startswith('Estimate')]
labels = labels [~labels.label.str.contains('ratio')]

groups = labels.label.str.split('!', expand=True)[2].unique()
groups

array(['EMPLOYMENT STATUS', 'COMMUTING TO WORK', 'OCCUPATION', 'INDUSTRY',
       'CLASS OF WORKER',
       'INCOME AND BENEFITS (IN 2021 INFLATION-ADJUSTED DOLLARS)',
       'HEALTH INSURANCE COVERAGE',
       'PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL'],
      dtype=object)

## Functions to combine everything

In [6]:
def cal_counts(label):
    #pull out estimate and moa
    name_est = label + 'e'
    name_moa = label + 'm'

    #find the corresponding title to census label
    string = labels [labels.column_name == name_est].label.iloc[0]
#     label_title = re.findall('!!([\w\s]+)$',string)[0]
    if not string.split('!!')[3:]:
        label_title = string.split('!!')[-1]
    else:
        label_title = '!!'.join(string.split('!!')[3:])

    #isolate just one indicator
    data = dataall[['geo_id', name_est, name_moa]].copy()

    #join districts and indicator together
    df = dist.merge(data, how='inner', right_on='geo_id', left_on='tract')
    df = df.replace(np.nan, 0)

    #get count percentages for each census tract per district
    df['tru_count'] = df[name_est].astype(float) * df.percent
    df['tru_error'] = df[name_moa].astype(float) * df.percent

    #for each council district, sum up counts, and calculate new moe
    council_counts = []
    total_counts = df.tru_count.sum()

    for x in df.council.unique():
        subset = df [df.council == x]

        count = round(subset.tru_count.sum(), 2)
        count_perc = round(count / total_counts, 2)

        error = round((subset.tru_error ** 2).sum()**.5, 2)
        error_perc = round(math.sqrt(sum((subset.tru_error / 1.645)**2)) / count, 2)

        council_counts.append([x, count, count_perc, error, error_perc])

    #convert to dataframe and format
    dff = pd.DataFrame(council_counts).sort_values(0)
    dff.columns = [label_title, 'counts', 'count_perc', 'moe', 'moe_perc']
    dff = dff.set_index(label_title, drop=True)
#     print(dff)

    return dff

In [7]:
def cal_district_numbers(labels_check):
    district_totals = pd.DataFrame(np.arange(11))
    totals = []

    for label in labels_check:
        dff = cal_counts(label)
        name = dff.index.name
        dff.columns = [name + ' counts', name + ' counts percent', name + ' moe', name + ' moa percent']
        dff = dff.reset_index(drop=True)

        district_totals = pd.concat([district_totals,dff],axis=1)

    district_totals = district_totals.drop(columns=0)
    return district_totals.T

## Now calculate! 

### Employment Status

In [8]:
grouping = groups[0]

current_labels = labels [labels.label.str.contains(grouping)]
current_labels = current_labels.iloc[:1]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)

Estimate!!EMPLOYMENT STATUS!!Population 16 years and over


In [9]:
final0 = cal_district_numbers(labels_check)
final0

,0,1,2,3,4,5,6,7,8,9,10
Population 16 years and over counts,404666.26,102401.21,108638.52,108029.32,97527.89,94468.13,127932.21,116881.10,136397.09,124085.79,118005.49
Population 16 years and over counts percent,0.26,0.07,0.07,0.07,0.06,0.06,0.08,0.08,0.09,0.08,0.08
Population 16 years and over moe,7035.94,2954.23,3296.56,3082.81,2941.07,3227.91,3943.50,3261.94,3185.89,3199.61,2733.26
Population 16 years and over moa percent,0.01,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.01,0.02,0.01


### Labor force

In [10]:
grouping = groups[0]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
current_labels = current_labels.iloc[1:]
# current_labels = current_labels.iloc[:-13]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)

EMPLOYMENT STATUS

Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force
Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Civilian labor force
Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Civilian labor force!!Employed
Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Civilian labor force!!Unemployed
Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!In labor force!!Armed Forces
Estimate!!EMPLOYMENT STATUS!!Population 16 years and over!!Not in labor force
Estimate!!EMPLOYMENT STATUS!!Civilian labor force
Estimate!!EMPLOYMENT STATUS!!Civilian labor force!!Unemployment Rate
Estimate!!EMPLOYMENT STATUS!!Females 16 years and over
Estimate!!EMPLOYMENT STATUS!!Females 16 years and over!!In labor force
Estimate!!EMPLOYMENT STATUS!!Females 16 years and over!!In labor force!!Civilian labor force
Estimate!!EMPLOYMENT STATUS!!Females 16 years and over!!In labor force!!Civilian labor force!

In [11]:
final1 = cal_district_numbers(labels_check)
final1

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in double_scalars


,0,1,2,3,4,5,6,7,8,9,10
In labor force counts,273048.17,64192.99,69007.17,63874.05,62205.32,51814.21,91292.00,74454.17,98828.80,86856.84,80986.29
In labor force counts percent,0.27,0.06,0.07,0.06,0.06,0.05,0.09,0.07,0.10,0.09,0.08
In labor force moe,5777.48,2290.67,2753.73,2229.22,2260.50,2073.86,3319.01,2578.96,2742.26,2749.15,2380.02
In labor force moa percent,0.01,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02
In labor force!!Civilian labor force counts,261107.84,63626.85,66000.60,63744.52,61378.08,51704.46,89596.00,73925.22,98001.89,85996.58,80031.95
...,...,...,...,...,...,...,...,...,...,...,...
Own children of the householder 6 to 17 years moa percent,0.02,0.05,0.04,0.04,0.04,0.04,0.04,0.05,0.04,0.04,0.04
All parents in family in labor force counts,67196.23,9485.10,15383.61,16343.47,17157.05,12818.81,20912.43,15388.19,13506.60,17558.81,15679.71
All parents in family in labor force counts percent,0.30,0.04,0.07,0.07,0.08,0.06,0.09,0.07,0.06,0.08,0.07
All parents in family in labor force moe,2922.22,1072.17,1372.49,1268.53,1437.53,1110.16,1726.57,1420.85,1206.03,1626.90,1356.95


### Commuting

In [12]:
grouping = groups[1]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
current_labels = current_labels.iloc[1:]
# current_labels = current_labels.iloc[:-16]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)

COMMUTING TO WORK

Estimate!!COMMUTING TO WORK!!Workers 16 years and over!!Car, truck, or van -- drove alone
Estimate!!COMMUTING TO WORK!!Workers 16 years and over!!Car, truck, or van -- carpooled
Estimate!!COMMUTING TO WORK!!Workers 16 years and over!!Public transportation (excluding taxicab)
Estimate!!COMMUTING TO WORK!!Workers 16 years and over!!Walked
Estimate!!COMMUTING TO WORK!!Workers 16 years and over!!Other means
Estimate!!COMMUTING TO WORK!!Workers 16 years and over!!Worked from home
Estimate!!COMMUTING TO WORK!!Workers 16 years and over!!Mean travel time to work (minutes)


In [13]:
final2 = cal_district_numbers(labels_check)    
final2

,0,1,2,3,4,5,6,7,8,9,10
"Car, truck, or van -- drove alone counts",192331.80,42864.98,46890.95,43373.66,41194.67,33831.50,62981.28,50997.83,67488.13,63277.30,59533.91
"Car, truck, or van -- drove alone counts percent",0.27,0.06,0.07,0.06,0.06,0.05,0.09,0.07,0.10,0.09,0.08
"Car, truck, or van -- drove alone moe",4116.03,1741.92,2339.34,1806.81,1653.74,1580.94,2499.41,1822.76,2153.61,2227.59,1978.30
"Car, truck, or van -- drove alone moa percent",0.01,0.02,0.03,0.03,0.02,0.03,0.02,0.02,0.02,0.02,0.02
"Car, truck, or van -- carpooled counts",26484.40,6779.25,7839.63,9612.87,9567.65,7508.67,10012.70,6934.56,8028.32,6127.52,7274.43
"Car, truck, or van -- carpooled counts percent",0.25,0.06,0.07,0.09,0.09,0.07,0.09,0.07,0.08,0.06,0.07
"Car, truck, or van -- carpooled moe",1877.51,784.56,989.99,1039.33,992.20,819.94,1188.98,735.25,997.48,721.37,886.69
"Car, truck, or van -- carpooled moa percent",0.04,0.07,0.08,0.07,0.06,0.07,0.07,0.06,0.08,0.07,0.07
Public transportation (excluding taxicab) counts,1621.31,1916.20,1993.73,1934.10,1334.23,1926.73,900.94,1616.99,1120.18,737.81,1077.77
Public transportation (excluding taxicab) counts percent,0.10,0.12,0.12,0.12,0.08,0.12,0.06,0.10,0.07,0.05,0.07


### Occupation

In [14]:
grouping = groups[2]
print(grouping)

current_labels = labels [labels.label.str.contains(grouping)]
current_labels = current_labels.iloc[1:]
# current_labels = current_labels.iloc[:-16]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)

OCCUPATION
Estimate!!OCCUPATION!!Civilian employed population 16 years and over!!Management, business, science, and arts occupations
Estimate!!OCCUPATION!!Civilian employed population 16 years and over!!Service occupations
Estimate!!OCCUPATION!!Civilian employed population 16 years and over!!Sales and office occupations
Estimate!!OCCUPATION!!Civilian employed population 16 years and over!!Natural resources, construction, and maintenance occupations
Estimate!!OCCUPATION!!Civilian employed population 16 years and over!!Production, transportation, and material moving occupations


In [15]:
final3 = cal_district_numbers(labels_check)
final3

,0,1,2,3,4,5,6,7,8,9,10
"Management, business, science, and arts occupations counts",109266.32,19277.67,14956.62,13831.96,12503.36,8166.35,30568.74,25730.32,43868.09,42331.36,30623.21
"Management, business, science, and arts occupations counts percent",0.31,0.05,0.04,0.04,0.04,0.02,0.09,0.07,0.12,0.12,0.09
"Management, business, science, and arts occupations moe",2848.34,1190.47,1069.02,930.76,874.11,773.19,1642.05,1282.45,1826.38,1736.42,1474.81
"Management, business, science, and arts occupations moa percent",0.02,0.04,0.04,0.04,0.04,0.06,0.03,0.03,0.03,0.02,0.03
Service occupations counts,37111.88,13741.15,15436.84,14577.58,13252.98,12944.96,15692.08,13054.43,15466.11,11027.87,12725.14
Service occupations counts percent,0.21,0.08,0.09,0.08,0.08,0.07,0.09,0.07,0.09,0.06,0.07
Service occupations moe,1890.02,978.62,1165.61,1079.27,1016.23,860.91,1258.94,1043.20,1302.65,1071.20,970.78
Service occupations moa percent,0.03,0.04,0.05,0.05,0.05,0.04,0.05,0.05,0.05,0.06,0.05
Sales and office occupations counts,55506.86,12899.04,13443.51,13578.49,14340.03,10165.83,19010.20,17770.78,21813.35,20448.72,18913.19
Sales and office occupations counts percent,0.25,0.06,0.06,0.06,0.07,0.05,0.09,0.08,0.10,0.09,0.09


### Industry

In [16]:
grouping = groups[3]
print(grouping)

current_labels = labels [labels.label.str.contains(grouping)]
current_labels = current_labels.iloc[1:]
# current_labels = current_labels.iloc[:-16]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)

INDUSTRY
Estimate!!INDUSTRY!!Civilian employed population 16 years and over!!Agriculture, forestry, fishing and hunting, and mining
Estimate!!INDUSTRY!!Civilian employed population 16 years and over!!Construction
Estimate!!INDUSTRY!!Civilian employed population 16 years and over!!Manufacturing
Estimate!!INDUSTRY!!Civilian employed population 16 years and over!!Wholesale trade
Estimate!!INDUSTRY!!Civilian employed population 16 years and over!!Retail trade
Estimate!!INDUSTRY!!Civilian employed population 16 years and over!!Transportation and warehousing, and utilities
Estimate!!INDUSTRY!!Civilian employed population 16 years and over!!Information
Estimate!!INDUSTRY!!Civilian employed population 16 years and over!!Finance and insurance, and real estate and rental and leasing
Estimate!!INDUSTRY!!Civilian employed population 16 years and over!!Professional, scientific, and management, and administrative and waste management services
Estimate!!INDUSTRY!!Civilian employed population 16 years

In [17]:
final4 = cal_district_numbers(labels_check)
final4

,0,1,2,3,4,5,6,7,8,9,10
"Agriculture, forestry, fishing and hunting, and mining counts",4141.84,443.24,887.32,764.06,410.62,203.77,1005.06,478.35,658.83,680.26,733.64
"Agriculture, forestry, fishing and hunting, and mining counts percent",0.40,0.04,0.09,0.07,0.04,0.02,0.10,0.05,0.06,0.07,0.07
"Agriculture, forestry, fishing and hunting, and mining moe",699.92,151.23,341.29,257.91,180.19,122.99,404.18,194.16,219.16,231.89,207.56
"Agriculture, forestry, fishing and hunting, and mining moa percent",0.10,0.21,0.23,0.21,0.27,0.37,0.24,0.25,0.20,0.21,0.17
Construction counts,16444.61,7060.11,6048.06,7113.87,7232.98,7824.54,5900.74,5759.91,4078.68,3949.60,5469.90
Construction counts percent,0.21,0.09,0.08,0.09,0.09,0.10,0.08,0.07,0.05,0.05,0.07
Construction moe,1423.62,773.76,809.75,760.61,769.07,884.22,1035.06,660.05,609.26,624.42,731.04
Construction moa percent,0.05,0.07,0.08,0.06,0.06,0.07,0.11,0.07,0.09,0.10,0.08
Manufacturing counts,14340.62,2673.99,3854.39,4390.74,3793.34,3610.24,4463.42,2997.31,3649.63,4174.75,3427.56
Manufacturing counts percent,0.28,0.05,0.08,0.09,0.07,0.07,0.09,0.06,0.07,0.08,0.07


### Class of worker

In [18]:
grouping = groups[4]
print(grouping)
print()

current_labels = labels [labels.label.str.contains(grouping)]
current_labels = current_labels.iloc[1:]
# current_labels = current_labels.iloc[:-16]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)

CLASS OF WORKER

Estimate!!CLASS OF WORKER!!Civilian employed population 16 years and over!!Private wage and salary workers
Estimate!!CLASS OF WORKER!!Civilian employed population 16 years and over!!Government workers
Estimate!!CLASS OF WORKER!!Civilian employed population 16 years and over!!Self-employed in own not incorporated business workers
Estimate!!CLASS OF WORKER!!Civilian employed population 16 years and over!!Unpaid family workers


In [19]:
final5 = cal_district_numbers(labels_check)
final5

,0,1,2,3,4,5,6,7,8,9,10
Private wage and salary workers counts,191281.02,49419.53,49735.18,47690.05,45454.81,38156.18,66219.51,54693.66,76603.51,65966.83,60150.73
Private wage and salary workers counts percent,0.26,0.07,0.07,0.06,0.06,0.05,0.09,0.07,0.10,0.09,0.08
Private wage and salary workers moe,4433.31,2136.76,2231.96,1971.72,1900.09,1688.63,2789.31,2062.64,2602.23,2328.48,2065.57
Private wage and salary workers moa percent,0.01,0.03,0.03,0.03,0.03,0.03,0.03,0.02,0.02,0.02,0.02
Government workers counts,41166.16,6251.28,7302.02,8544.81,7615.99,5269.40,13135.51,10471.12,11987.33,11018.83,11035.55
Government workers counts percent,0.31,0.05,0.05,0.06,0.06,0.04,0.10,0.08,0.09,0.08,0.08
Government workers moe,1999.60,605.04,931.75,737.08,766.95,633.26,1093.37,854.04,943.99,881.28,921.92
Government workers moa percent,0.03,0.06,0.08,0.05,0.06,0.07,0.05,0.05,0.05,0.05,0.05
Self-employed in own not incorporated business workers counts,15491.72,4384.81,3870.85,3958.51,3600.26,4057.14,3861.29,4376.27,3958.96,5600.57,4659.61
Self-employed in own not incorporated business workers counts percent,0.27,0.08,0.07,0.07,0.06,0.07,0.07,0.08,0.07,0.10,0.08


### Income and benefits

In [20]:
grouping = groups[5][:19]
print(grouping)

current_labels = labels [labels.label.str.contains(grouping)]
# current_labels = current_labels.iloc[:-2]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)

INCOME AND BENEFITS
Estimate!!INCOME AND BENEFITS (IN 2021 INFLATION-ADJUSTED DOLLARS)!!Total households
Estimate!!INCOME AND BENEFITS (IN 2021 INFLATION-ADJUSTED DOLLARS)!!Total households!!Less than $10,000
Estimate!!INCOME AND BENEFITS (IN 2021 INFLATION-ADJUSTED DOLLARS)!!Total households!!$10,000 to $14,999
Estimate!!INCOME AND BENEFITS (IN 2021 INFLATION-ADJUSTED DOLLARS)!!Total households!!$15,000 to $24,999
Estimate!!INCOME AND BENEFITS (IN 2021 INFLATION-ADJUSTED DOLLARS)!!Total households!!$25,000 to $34,999
Estimate!!INCOME AND BENEFITS (IN 2021 INFLATION-ADJUSTED DOLLARS)!!Total households!!$35,000 to $49,999
Estimate!!INCOME AND BENEFITS (IN 2021 INFLATION-ADJUSTED DOLLARS)!!Total households!!$50,000 to $74,999
Estimate!!INCOME AND BENEFITS (IN 2021 INFLATION-ADJUSTED DOLLARS)!!Total households!!$75,000 to $99,999
Estimate!!INCOME AND BENEFITS (IN 2021 INFLATION-ADJUSTED DOLLARS)!!Total households!!$100,000 to $149,999
Estimate!!INCOME AND BENEFITS (IN 2021 INFLATION-ADJUS

In [21]:
final6 = cal_district_numbers(labels_check)
final6

,0,1,2,3,4,5,6,7,8,9,10
Total households counts,177329.79,49774.31,51222.14,49100.15,40884.53,40961.91,57962.61,56476.88,69755.57,64733.36,58922.75
Total households counts percent,0.25,0.07,0.07,0.07,0.06,0.06,0.08,0.08,0.10,0.09,0.08
Total households moe,2512.64,1355.11,1382.56,1229.05,990.80,1169.42,1486.44,1416.77,1552.83,1487.35,1194.84
Total households moa percent,0.01,0.02,0.02,0.02,0.01,0.02,0.02,0.02,0.01,0.01,0.01
"Less than $10,000 counts",5366.70,4580.61,5415.12,5380.55,2477.68,5490.16,2270.53,5278.03,4967.44,2356.20,2554.99
...,...,...,...,...,...,...,...,...,...,...,...
"Median earnings for male full-time, year-round workers (dollars) moa percent",0.02,0.03,0.03,0.03,0.02,0.02,0.03,0.03,0.03,0.03,0.03
"Median earnings for female full-time, year-round workers (dollars) counts",3847624.09,1199918.18,1085309.73,995207.73,750023.90,868868.10,1112887.37,1207782.43,1433130.97,1608279.78,1337030.72
"Median earnings for female full-time, year-round workers (dollars) counts percent",0.25,0.08,0.07,0.06,0.05,0.06,0.07,0.08,0.09,0.10,0.09
"Median earnings for female full-time, year-round workers (dollars) moe",131840.34,61642.24,62186.62,53440.60,39256.82,33944.04,56703.13,48846.97,57153.77,64697.65,56461.37


### Health Insurance Coverage

In [22]:
grouping = groups[6]
print(grouping)

current_labels = labels [labels.label.str.contains(grouping)]
# current_labels = current_labels.iloc[:-2]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)

HEALTH INSURANCE COVERAGE
Estimate!!HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population
Estimate!!HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population!!With health insurance coverage
Estimate!!HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population!!With health insurance coverage!!With private health insurance
Estimate!!HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population!!With health insurance coverage!!With public coverage
Estimate!!HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population!!No health insurance coverage
Estimate!!HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population under 19 years
Estimate!!HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population under 19 years!!No health insurance coverage
Estimate!!HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population 19 to 64 years
Estimate!!HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population 19 to 64 years!!In

In [23]:
final7 = cal_district_numbers(labels_check)
final7

,0,1,2,3,4,5,6,7,8,9,10
Civilian noninstitutionalized population counts,522531.05,123353.54,139375.51,139651.49,131328.75,121988.46,166066.43,145755.40,164924.96,153865.12,147755.28
Civilian noninstitutionalized population counts percent,0.27,0.06,0.07,0.07,0.07,0.06,0.08,0.07,0.08,0.08,0.08
Civilian noninstitutionalized population moe,8575.67,3764.04,4442.13,4026.04,4157.12,4157.79,5755.08,4424.54,4046.24,4726.86,3844.10
Civilian noninstitutionalized population moa percent,0.01,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.01,0.02,0.02
With health insurance coverage counts,462247.94,93371.67,113059.15,111469.48,104626.34,91958.02,141889.38,122223.76,143268.62,137771.89,128609.75
...,...,...,...,...,...,...,...,...,...,...,...
Not in labor force:!!With health insurance coverage!!With public coverage moa percent,0.05,0.06,0.05,0.05,0.07,0.06,0.07,0.07,0.08,0.09,0.07
Not in labor force:!!No health insurance coverage counts,12911.71,7552.84,6126.82,7543.12,5667.56,7794.01,4381.64,5727.05,4085.71,2612.34,3792.19
Not in labor force:!!No health insurance coverage counts percent,0.19,0.11,0.09,0.11,0.08,0.11,0.06,0.08,0.06,0.04,0.06
Not in labor force:!!No health insurance coverage moe,1120.82,911.65,812.03,907.77,632.42,727.40,638.68,859.90,664.51,458.79,526.94


### Below poverty level

In [24]:
grouping = groups[7]
print(grouping)

current_labels = labels [labels.label.str.contains(grouping)]
# current_labels = current_labels.iloc[:-2]
labels_check = current_labels.column_name.str[:-1]

for titles in current_labels.label:
    print(titles)

PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL
Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!All families
Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!All families!!With related children of the householder under 18 years
Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!All families!!With related children of the householder under 18 years!!With related children of the householder under 5 years only
Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!All families!!Married couple families
Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!All families!!Married couple families!!With related children of the householder under 18 years
Estimate!!PERCENTAGE O

In [25]:
final8 = cal_district_numbers(labels_check)
final8

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in double_scalars


,0,1,2,3,4,5,6,7,8,9,10
All families counts,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
All families counts percent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
All families moe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
All families moa percent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
With related children of the householder under 18 years counts,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
People in families moa percent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Unrelated individuals 15 years and over counts,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Unrelated individuals 15 years and over counts percent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Unrelated individuals 15 years and over moe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
with pd.ExcelWriter('sa2020_ccd_demo.xlsx') as writer:
    final0.to_excel(writer, sheet_name='Total Population')
    final1.to_excel(writer, sheet_name=groups[0][:30])
    final2.to_excel(writer, sheet_name=groups[1][:30])
    final3.to_excel(writer, sheet_name=groups[2][:30])
    final4.to_excel(writer, sheet_name=groups[3][:30])
    final5.to_excel(writer, sheet_name=groups[4][:30])
    final6.to_excel(writer, sheet_name=groups[5][:30])
    final7.to_excel(writer, sheet_name=groups[6][:30])
    final8.to_excel(writer, sheet_name=groups[7][:30])